In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import time
import os
import cv2
import math
# import pafy
import random
import numpy as np
import datetime as dt
import tensorflow as tf
# from moviepy.editor import *
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [2]:
seed_constant = 23
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [3]:
image_height, image_width = 64, 64
max_images_per_class = 6000

dataset_directory = "HandWashDataset"
classes_list = ['Step_1', 'Step_2_Left', 'Step_2_Right', 'Step_3', 
                'Step_4_Left', 'Step_4_Right', 'Step_5_Left', 'Step_5_Right', 
                'Step_6_Left', 'Step_6_Right', 'Step_7_Left', 'Step_7_Right']

model_output_size = len(classes_list)

In [4]:
# load array
features = np.load('../input/preprocess-dataset-v2/features v2.npz')
features  = features['arr_0']

labels = np.load('../input/preprocess-dataset-v2/labels v2.npy')

In [5]:
# Convert labels
one_hot_encoded_labels = to_categorical(labels) 

In [6]:
# Split Dataset into train and test
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.2, shuffle = True, random_state = seed_constant)

In [7]:
del features

In [8]:
# Function that will construct the model
def create_model():
    model = Sequential()
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', input_shape = (image_height, image_width, 3)))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Dropout(0.5))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(model_output_size, activation = 'softmax'))

    model.summary()

    return model


model = create_model()

print("Model Created Successfully!")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 128)       73856     
_________________________________________________________________
batch_normalization (BatchNo (None, 29, 29, 128)       512       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 128)       0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 128)               0

In [9]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 3, mode = 'min', restore_best_weights = True)
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

model_training_history = model.fit(x = features_train, y = labels_train, epochs = 50, batch_size = 4 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

Epoch 1/50
11520/11520 [==============================] - 386s 33ms/step - loss: 2.1548 - accuracy: 0.2440 - val_loss: 0.7695 - val_accuracy: 0.7123
Epoch 2/50
11520/11520 [==============================] - 387s 34ms/step - loss: 0.9637 - accuracy: 0.6709 - val_loss: 0.4772 - val_accuracy: 0.8366
Epoch 3/50
11520/11520 [==============================] - 386s 34ms/step - loss: 0.6612 - accuracy: 0.7800 - val_loss: 0.3660 - val_accuracy: 0.8850
Epoch 4/50
11520/11520 [==============================] - 390s 34ms/step - loss: 0.5269 - accuracy: 0.8277 - val_loss: 0.4633 - val_accuracy: 0.8634
Epoch 5/50
11520/11520 [==============================] - 388s 34ms/step - loss: 0.4732 - accuracy: 0.8487 - val_loss: 0.1977 - val_accuracy: 0.9490
Epoch 6/50
11520/11520 [==============================] - 388s 34ms/step - loss: 0.4272 - accuracy: 0.8647 - val_loss: 0.2382 - val_accuracy: 0.9361
Epoch 7/50
11520/11520 [==============================] - 390s 34ms/step - loss: 0.3967 - accuracy: 0.8730

In [10]:
from keras.models import model_from_json
from keras.models import load_model

# serialize model to JSON
#  the keras model which is trained is defined as 'model' in this example
model_json = model.to_json()


with open("model_num.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model_num.h5") #namainnya sabeb kan wkwk

In [ ]:
#ga pake cell ini 
#save model
date_time_format = '%Y_%m_%d__%H_%M_%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)
model_evaluation_loss , model_evaluation_accuracy = model_evaluation_history 
model_name = model_name = f'./Model__Date_Time{current_date_time_string}__Loss{model_evaluation_loss}__Accuracy{model_evaluation_accuracy}.h5'

model.save(model_name)